In [82]:
import pandas as pd
import os
import sys
from pathlib import Path  # Melhor forma para trabalhar com caminhos


### Convertendo xlsx para csv


In [87]:
pasta_origem = r'E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_02_DATASETS\datasets_original'
pasta_destino = r"E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_03_ANALISE_EXPLORATORIA"

for arquivo in os.listdir(pasta_origem):
    if arquivo.endswith('.xlsx'):
        entrada = os.path.join(pasta_origem, arquivo)
        saida = os.path.join(pasta_destino, arquivo.replace('.xlsx', '.csv'))
        try:
            df = pd.read_excel(entrada, header=1, engine='openpyxl')
            df.to_csv(saida, index=False)
            print(f"Convertido: {arquivo}")
        except Exception as e:
            print(f"Erro ao converter {arquivo}: {e}")


Convertido: Balanço Patrimonial (BP).xlsx
Convertido: Demonstração de Fluxo de Caixa (DFC).xlsx
Convertido: Demonstração de Resultados (DRE).xlsx


### Pivotando arquivos CSV

In [93]:


def process_files(input_folder, output_folder):
    """Processa todos os arquivos CSV na pasta de entrada e salva na pasta de saída"""
    # Garante que a pasta de saída existe
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    

    for arquivo in os.listdir(input_folder):
        if arquivo.endswith('.csv') or arquivo.endswith('.xlsx'):
            try:
                nome = os.path.splitext(arquivo)[0]
                file_path = os.path.join(input_folder, arquivo)
                #trasnformar arquivo para csv e retirar a prima linha
                if arquivo.endswith('.xlsx'):
                    df = pd.read_excel(file_path, header=1)
                    #cria um  nome temporário para o CSV
                    temp_csv_path = os.path.join(input_folder, f"temp_{nome}.csv")
                    df.to_csv(temp_csv_path, index=False, encoding='utf-8')
                    file_path = temp_csv_path
                # Ler o CSV
                df = pd.read_csv(file_path, encoding='utf-8')
                
                # Transformar de wide para long
                df_long = df.melt(
                    id_vars=['Código da Conta','Descrição da Conta'],
                    var_name='data',
                    value_name='valor'
                )
           
                # Tratamento de nulos
                df_long = df_long.assign(valor =lambda x: x['valor'].fillna(0),
                                         codigo_conta = lambda x: x['Código da Conta'].fillna(0),
                                         descricao_conta = lambda x: x['Descrição da Conta'].fillna(0))
                # Ordenação
                df_long = df_long.sort_values(['Código da Conta','data'])
                
                # Salvar arquivo processado
                output_file = f"{nome}_rotacionado.csv"
                output_path = os.path.join(output_folder, output_file)
                df_long.to_csv(output_path, index=False, encoding='utf-8')
                
                print(f"Arquivo {arquivo} processado com sucesso! Salvo como {output_file}")
                print(df_long.head())
                
            except Exception as e:
                print(f"Erro ao processar o arquivo {arquivo}: {str(e)}")

# Configuração dos caminhos (usando Path para compatibilidade entre sistemas)
base_dir = Path(r"E:\PYTHON\PROJETOS_PESSOAIS\analise_weg")
input_folder = base_dir / "_03_ANALISE_EXPLORATORIA\_03_01_dataset_convertido"
output_folder = base_dir / "_04_ETL/datasets_pivotados"

# Executar o processamento
process_files(input_folder, output_folder)

Arquivo Balanço Patrimonial (BP).csv processado com sucesso! Salvo como Balanço Patrimonial (BP)_rotacionado.csv
    Código da Conta Descrição da Conta                 data  valor  \
44                                 NaN  2010-03-31 00:00:00    0.0   
45                                 NaN  2010-03-31 00:00:00    0.0   
153                                NaN  2010-06-30 00:00:00    0.0   
154                                NaN  2010-06-30 00:00:00    0.0   
262                                NaN  2010-09-30 00:00:00    0.0   

    codigo_conta descricao_conta  
44                             0  
45                             0  
153                            0  
154                            0  
262                            0  
Arquivo Demonstração de Fluxo de Caixa (DFC).csv processado com sucesso! Salvo como Demonstração de Fluxo de Caixa (DFC)_rotacionado.csv
    Código da Conta                     Descrição da Conta  \
0              6.01  Caixa Líquido Atividades Operacionai

<>:49: SyntaxWarning: invalid escape sequence '\_'
<>:49: SyntaxWarning: invalid escape sequence '\_'
C:\Users\gabri\AppData\Local\Temp\ipykernel_1560\758981859.py:49: SyntaxWarning: invalid escape sequence '\_'
  input_folder = base_dir / "_03_ANALISE_EXPLORATORIA\_03_01_dataset_convertido"


In [90]:
df = pd.read_csv(r"E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_03_ANALISE_EXPLORATORIA\_03_02_DATASETS_PIVOTADOS\Balanço Patrimonial (BP)_rotacionado.csv")

In [ ]:
df.head(100)  # Exibir as primeiras linhas do DataFrame para verificação